## Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns




## Reading the data

In [2]:
df_food = pd.read_csv('food.csv')
df_rating = pd.read_csv('ratings.csv')
df_food.head(5)
#df_rating.head(5)



,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [3]:
df_food.shape

(309, 5)

In [4]:
# Pivot table
dataset = df_rating.pivot_table(index='Food_ID', columns='User_ID', values='Rating')
dataset.fillna(0, inplace=True)


In [27]:
def get_food_by_column(column_name, quantity_name):
    if column_name == 'Name':
        return df_food[df_food[column_name].str.contains(quantity_name)]
    elif column_name == 'Muscle Gain':
        return df_food[(df_food['C_Type'] == 'Healthy Food') & (df_food['Veg_Non'] == 'non-veg').any()]
    else:
        print("In case you want to add more field filters")


def predict_food(FoodList, FOODS_TO_PREDICT = 10):
    if len(FoodList):
        Foodi= FoodList.iloc[0]['Food_ID']
        Foodi = dataset[dataset['Food_ID'] == Foodi].index[0]
        distances , indices = model.kneighbors(csr_dataset[Foodi],n_neighbors=FOODS_TO_PREDICT+1)    
        Food_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        Recommendations = []

        for val in Food_indices:
            Foodi = dataset.iloc[val[0]]['Food_ID']
            i = df_food[df_food['Food_ID'] == Foodi].index
            Recommendations.append({'Name':df_food.iloc[i]['Name'].values[0],'Distance':val[1]})

        df = pd.DataFrame(Recommendations,index=range(1,FOODS_TO_PREDICT+1))
        return df['Name']
    else:
        return "Sorry, we cannot find this food in our database"

## Main Recommendation Function 

In [34]:
def food_recommendation(Food_Name, CategoryName):
    if Food_Name:
        FoodList = get_food_by_column('Name', Food_Name)
        return predict_food(FoodList)
    elif CategoryName:
        FoodList = get_food_by_column('Muscle Gain', None)
        return predict_food(FoodList)
    else:
        print("In case you want to add more model input options")

In [14]:
csr_dataset = csr_matrix(dataset.values)
dataset.reset_index(inplace=True)

In [15]:
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model.fit(csr_dataset)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [29]:
food_recommendation(None, 'Muscle Gain')

1                                    corn pulao
2                            chocolate doughnut
3                      andhra pan fried pomfret
4         surmai curry with lobster butter rice
5                                   apple rabdi
6                                chocolate appe
7                       grilled lemon margarita
8                              french pork chop
9     japanese curry arancini with barley salsa
10                       chocolate nero cookies
Name: Name, dtype: object

In [35]:
# For food based prediction
food_recommendation('chicken minced salad', None)

1                                   egg in a blanket
2             cajun spiced turkey wrapped with bacon
3                  caramelized sesame smoked almonds
4                                     chocolate appe
5                                     chicken paella
6                  roast turkey with cranberry sauce
7                            vegetable som tam salad
8                                   french pork chop
9     couscous with ratatouille - tangy tomato sauce
10                                    christmas cake
Name: Name, dtype: object

In [1]:
import pickle

pickle.dump(model, open('model.pkl','wb'))

NameError: name 'model' is not defined